In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title:* Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)


In [ ]:
##REMOVE EMPLOYMENT RATE DATA PER CONVERSATION WITH DOL R&A STAFF-- SCRIPT INITIALLY RUN, BUT TIMED OUT, FIND REMAINING
##STORYMAPS THAT DIDN'T RECEIVE UPDATE (EZH -- 7/3/2019)
upd_lst = []
for f in find_storymaps:
    if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communities ') and not f.title.startswith('Community '):
        csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
        sections = csmap.properties["values"]["story"]["sections"]
        panel_econ = sections[7]
        try:
            if str('Employment') in str(panel_econ["content"]):
                upd_lst.append(f)
        except Exception as e:
            print(f.title, str(e))
            continue
            
print(upd_lst)
        

In [ ]:
for f in upd_lst:
    if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communities ') and not f.title.startswith('Community '):
        csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
        sections = csmap.properties["values"]["story"]["sections"]
        panel_econ = sections[7]
        community = f.title.replace(" Community Storymap", "")
        if community.startswith("City & Borough of "):
            community = community.replace("City & Borough of ", "")
        if community.startswith("Municipality of "):
            community = community.replace("Municipality of ","")
        query = "CommunityName = '" + str(community.replace("'","''")) + "'"
      
    ### ECONOMY PANEL ###
        #Taxes data
        print(f.title, f.id)
        tax_base = gis.content.get("70a364eae3514531a280e4696b54c6e9").layers[0]
        tax_data = tax_base.query(where="CommunityName = '" + str(community.replace("'","''")) + "' AND RevenueYear = 2016", out_fields = "*").df
        if str(tax_data) != 'Empty DataFrame\nColumns: []\nIndex: []':
            sales_tax = tax_data.SalesTaxPercentage[0]
            prop_tax = tax_data.PropertyTaxMills[0]
            other_tax = tax_data.OtherTaxes[0]
            prop_tax_rev = tax_data.PropertyTaxRevenue[0]
            sales_tax_rev = tax_data.SalesTaxRevenue[0]
            total_tax_rev = tax_data.TotalTaxRevenue[0]
            tax_table = ['<table border="1"><tr><th width="40%"></th><th width="60%"></th></tr>',
                         '<tr><td>Property Tax Mills:</td><td>' + str(prop_tax) + '</td></tr>',
                        '<tr><td>Sales Tax Rate: </td><td>' + str(sales_tax) + '</td></tr>',
                        '<tr><td>Other Taxes: </td><td>' + str(other_tax) + '</td></tr>',
                        '<tr><td>Total Tax Revenue: </td><td> $' + str("{:,}".format(total_tax_rev)) + '</td></tr>',
                        '<tr><td>Sales Tax Revenue: </td><td> $' + str("{:,}".format(sales_tax_rev)) + '</td></tr>']
            embed_tax_tbl = ' '.join(tax_table)
        else:
            embed_tax_tbl = '<i> There is no tax data currently available for this community, or this community does not assess taxes.</i>'

        #Income and poverty data
        inc_pov_base = gis.content.get("ad27d89db59744b89766e8131af0c13d")
        inc_pov_data = inc_pov_base.layers[0].query(where=str(query) + " AND IsMostRecent = 1", out_fields = "*").df
        if str(inc_pov_data) != 'Empty DataFrame\nColumns: []\nIndex: []':
            hs_inc = inc_pov_data.MedianHouseholdIncome[0]
            fm_inc = inc_pov_data.MedianFamilyIncome[0]
            poverty = inc_pov_data.PersonsBelowPoveryLevel[0]
            poverty125 = inc_pov_data.PersonsBelow125PercentOfPovLev[0]
            if str(hs_inc) != 'None' and str(fm_inc) != 'None':
                inc_pov_tbl = ['<table border="1"><tr><th width="70%"></th><th width="30%"></th></tr>',
                             '<tr><td>Median Household Income:</td><td> $' + str("{:,}".format(hs_inc)) + '</td></tr>',
                            '<tr><td>Median Family Income: </td><td> $' + str("{:,}".format(fm_inc)) + '</td></tr>',
                            '<tr><td>Persons Below Poverty: </td><td>' + str(poverty) + '</td></tr>',
                            '<tr><td>Persons Below 125% of the Poverty Level: </td><td> ' + str(poverty125) + '</td></tr></table>']
            else:
                inc_pov_tbl = ['<table border="1"><tr><th width="70%"></th><th width="30%"></th></tr>',
                             '<tr><td>Median Household Income:</td><td> $' + str(hs_inc) + '</td></tr>',
                            '<tr><td>Median Family Income: </td><td> $' + str(fm_inc) + '</td></tr>',
                            '<tr><td>Persons Below Poverty: </td><td>' + str(poverty) + '</td></tr>',
                            '<tr><td>Persons Below 125% of the Poverty Level: </td><td> ' + str(poverty125) + '</td></tr></table>']
            embed_IncPov_tbl = ' '.join(inc_pov_tbl)
        else:
            embed_IncPov_tbl = '<i> There is no income and poverty data currently available for this community. </i>'


        panel_econ["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div><p><b><u>Taxes: </u></b> ' + str(embed_tax_tbl) + '</table></p></div><p>&nbsp;</p><p><b><u>Income and Poverty:</u></b>' + str(embed_IncPov_tbl) + '</p>\n\n<p>&nbsp;</p><p>More information can be found in the <a href="http://DCCED.maps.arcgis.com/apps/MapJournal/index.html?appid=bb631449256346db81bc026339f0e60c" target="_blank">Alaska Taxable.</a></p>\n'

        csmap.save()
        print("Economic data posted to economy panel.")